# Analyse des données et prédiction par la moyenne de chaque arrondissment

In [1]:
# Importing the librairies
import pandas as pd

# Exporting the csv file into a DataFrame pandas
chemin_fichier = "../data/data_cleaned.csv"
df = pd.read_csv(chemin_fichier)

In [2]:
# Conversion en datetime si ce n'est pas déjà fait
df['date_mutation'] = pd.to_datetime(df['date_mutation'])

# Extraire seulement l'année (en tant qu'entier)
df['année'] = df['date_mutation'].dt.year


## Prédiction du prix par arrondissement (on prédit seulement la moyenne)

In [3]:
from sklearn.model_selection import train_test_split

# séparation des données en train test (directement appliquable sur le dataframe)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
from collections import defaultdict

Moyenne_prix_au_m2 = defaultdict(float)
Compte_appartements = defaultdict(int)

# calcul des moyennes par arrondissement
for index, row in train_df.iterrows():
    arrondissement = int(row['arrondissement'])
    prix = row['prix_au_m2']
    
    Moyenne_prix_au_m2[arrondissement] += prix
    Compte_appartements[arrondissement] += 1

# Calcul de la moyenne
for arr in Moyenne_prix_au_m2:
    Moyenne_prix_au_m2[arr] /= Compte_appartements[arr]

In [5]:
# affichage de ces moyennes
for key, value in sorted(Moyenne_prix_au_m2.items(), key=lambda x: x[0]):
    print(f"Arrondissement n°{key} --> prix moyen au m² : {value:.2f} €")

Arrondissement n°1 --> prix moyen au m² : 14470.95 €
Arrondissement n°2 --> prix moyen au m² : 12540.24 €
Arrondissement n°3 --> prix moyen au m² : 12904.99 €
Arrondissement n°4 --> prix moyen au m² : 14148.98 €
Arrondissement n°5 --> prix moyen au m² : 12893.60 €
Arrondissement n°6 --> prix moyen au m² : 16022.65 €
Arrondissement n°7 --> prix moyen au m² : 15743.10 €
Arrondissement n°8 --> prix moyen au m² : 13447.64 €
Arrondissement n°9 --> prix moyen au m² : 11806.54 €
Arrondissement n°10 --> prix moyen au m² : 10536.83 €
Arrondissement n°11 --> prix moyen au m² : 10997.67 €
Arrondissement n°12 --> prix moyen au m² : 9945.05 €
Arrondissement n°13 --> prix moyen au m² : 9726.83 €
Arrondissement n°14 --> prix moyen au m² : 10608.49 €
Arrondissement n°15 --> prix moyen au m² : 10676.90 €
Arrondissement n°16 --> prix moyen au m² : 12169.36 €
Arrondissement n°17 --> prix moyen au m² : 11249.27 €
Arrondissement n°18 --> prix moyen au m² : 10089.11 €
Arrondissement n°19 --> prix moyen au m

#### Erreurs sur les données de test

In [6]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# prédiction par la moyenne de chaque arrondissement
y_test, y_pred = [],[]

for index, row in test_df.iterrows():
    y_test.append(row['prix_au_m2'])
    y_pred.append(Moyenne_prix_au_m2[row['arrondissement']])

# Métriques
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Affichage
print(f"MAE (erreur absolue moyenne) : {mae:.2f}")
print(f"RMSE (racine de l’erreur quadratique moyenne) : {rmse:.2f}")
print(f"R² (coefficient de détermination) : {r2:.4f}")

MAE (erreur absolue moyenne) : 1913.02
RMSE (racine de l’erreur quadratique moyenne) : 3322.48
R² (coefficient de détermination) : 0.2006


#### Erreurs très conséquentes. Se tromper de 4k € pour environ 10-15k € du m2 est très grand. De même R2 légèrement positif nous indique que ce modèle est quand même plus performant que juste prédire la baseline

#### Traçons maintenant un grahique pour voir la répartition géographique des maisons par rapport à leurs prix au m2

In [7]:
import plotly.express as px

fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    width=600,
    height=600,
    color='prix_au_m2',
    hover_data=["prix_au_m2"],
    range_color=(0, 30000),
    color_continuous_scale='Inferno_r' 
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

/var/folders/g9/1z_9t2sn34d6xr4x1kfly2dw0000gn/T/ipykernel_56572/3050270254.py:3: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


In [8]:
import json

# 1. Création du DataFrame 
df_moyenne = pd.DataFrame([
    {"c_arinsee": 75100 + int(arr), "prix_au_m2": prix}
    for arr, prix in Moyenne_prix_au_m2.items()
])

# 2. Chargement du fichier GeoJSON qui contient les coordonnées des arrondissments
with open("arrondissements.geojson", "r", encoding="utf-8") as f:
    geojson = json.load(f)

# 3. Création de la carte 
fig = px.choropleth_mapbox(
    df_moyenne,
    geojson=geojson,
    locations="c_arinsee",                 
    featureidkey="properties.c_arinsee",     
    color="prix_au_m2",
    color_continuous_scale="YlOrRd",
    range_color=(0, 20000),
    mapbox_style="open-street-map",
    zoom=11,
    center={"lat": 48.8566, "lon": 2.3522},
    opacity=0.6,
    width=800,
    height=600,
    labels={"prix_au_m2": "Prix moyen au m²"}
)

fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()


/var/folders/g9/1z_9t2sn34d6xr4x1kfly2dw0000gn/T/ipykernel_56572/1938064240.py:14: DeprecationWarning:

*choropleth_mapbox* is deprecated! Use *choropleth_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



#### Quand on compare cette carte avec des données trouvées sur internet on remarque plusieurs choses. D'abbord on a un effet périphérique légèrement moins prononcé, on a des plus grandes variations de prix vers le centre (à l'exception du 16e arrondissement). 